In [3]:
!pip install requests --upgrade --quiet

In [4]:
import requests

In [5]:
#scrape the webpages

In [6]:
fcs_data_p1_url = 'https://www.ncaa.com/stats/football/fcs/current/individual/469'

In [7]:
fcs_data_p2_url = 'https://www.ncaa.com/stats/football/fcs/current/individual/469/p2'

In [8]:
fcs_data_p3_url = 'https://www.ncaa.com/stats/football/fcs/current/individual/469/p3'

In [9]:
fcs_data_p4_url = 'https://www.ncaa.com/stats/football/fcs/current/individual/469/p4'

In [10]:
#name response variables for each webpage

In [11]:
resp1 = requests.get(fcs_data_p1_url)

In [12]:
resp2 = requests.get(fcs_data_p2_url)

In [13]:
resp3 = requests.get(fcs_data_p3_url)

In [14]:
resp4 = requests.get(fcs_data_p4_url)

In [15]:
#display all status codes so we know the responses are valid

In [16]:
resp1.status_code

200

In [17]:
resp2.status_code

200

In [18]:
resp3.status_code

200

In [19]:
resp4.status_code

200

In [20]:
#i'd like to loop through the status codes and only print something if the code isn't 200-299

In [21]:
#name page content variables that point to the text

In [22]:
fcs_pc1 = resp1.text

In [23]:
fcs_pc2 = resp2.text

In [24]:
fcs_pc3 = resp3.text

In [25]:
fcs_pc4 = resp4.text

In [26]:
#idk if i need this part, but I'm leaving it in for now

In [27]:
#with open('webpage.html', 'w') as f:
    #f.write(page_contents)

In [28]:
#get beautifulsoup

In [29]:
!pip install beautifulsoup4 --upgrade --quiet

In [30]:
from bs4 import BeautifulSoup

In [31]:
#name 4 bs docs that carry the page contents

In [32]:
fcs_doc1 = BeautifulSoup(fcs_pc1, 'html.parser')

In [33]:
fcs_doc2 = BeautifulSoup(fcs_pc2, 'html.parser')

In [34]:
fcs_doc3 = BeautifulSoup(fcs_pc3, 'html.parser')

In [35]:
fcs_doc4 = BeautifulSoup(fcs_pc4, 'html.parser')

In [36]:
#get the table row tags from each doc

In [40]:
tr_tags1 = fcs_doc1.find_all('tr')

In [41]:
tr_tags2 = fcs_doc2.find_all('tr')

In [42]:
tr_tags3 = fcs_doc3.find_all('tr')

In [43]:
tr_tags4 = fcs_doc4.find_all('tr')

In [44]:
#get ast

In [45]:
import ast

In [46]:
#get pandas

In [47]:
import pandas as pd

In [48]:
#name df columns

In [49]:
columns = ['Rank', 'Name', 'Team', 'Cl', 'Position','G', 'Rush', 'Rush TD', 'Rush Yds']

In [50]:
#make a function to get the string list into a workable format

In [51]:
def pull_out_row(tr_tag_row):
    new_info = str(tr_tag_row)
    new_info = new_info.replace('"', '')
    new_info = new_info.replace('<td>','"')
    new_info = new_info.replace('</td>','",')
    new_info = new_info.replace('<tr>', '[')
    new_info = new_info.replace('</tr>', ']')
    new_info = new_info.replace(',\n]', ']')
    new_info = ast.literal_eval(new_info)
    
    new_info = pd.DataFrame(new_info).T
    
    return new_info

In [52]:
#create a function to create a table from all of our formatted rows

In [53]:
def parse_table(table, columns):
    table = table[1:]
    
    master_df = pull_out_row(table.pop(0))

    for i in table:
        temp = pull_out_row(i)
        master_df = master_df.append(temp, ignore_index = True)

    master_df.columns = columns

    return master_df

In [54]:
df1 = parse_table(tr_tags1, columns)

In [55]:
df2 = parse_table(tr_tags2, columns)

In [56]:
df3 = parse_table(tr_tags3, columns)

In [57]:
df4 = parse_table(tr_tags4, columns)

In [62]:
bigtable_fcs = df1.append(df2, ignore_index = True)

In [63]:
bigtable_fcs = bigtable.append(df3, ignore_index = True)

In [64]:
bigtable_fcs = bigtable.append(df4, ignore_index = True)

In [65]:
bigtable_fcs

,Rank,Name,Team,Cl,Position,G,Rush,Rush TD,Rush Yds
0,1,Pierre Strong,<img src=https://i.turner.ncaa.com/sites/defau...,Sr.,RB,15,240,18,1686
1,2,Quay Holmes,<img src=https://i.turner.ncaa.com/sites/defau...,Sr.,RB,13,265,17,1553
2,3,Isaiah Ifanse,<img src=https://i.turner.ncaa.com/sites/defau...,Jr.,RB,13,262,10,1539
3,4,Malik Grant,<img src=https://i.turner.ncaa.com/sites/defau...,Jr.,RB,12,240,9,1347
4,5,Ramon Jefferson,<img src=https://i.turner.ncaa.com/sites/defau...,Sr.,RB,12,173,13,1155
...,...,...,...,...,...,...,...,...,...
245,-,Uriah West,<img src=https://i.turner.ncaa.com/sites/defau...,Jr.,RB,6,57,2,373
246,197,Taron Jones,<img src=https://i.turner.ncaa.com/sites/defau...,Jr.,RB,13,85,6,370
247,-,Owen Wright,<img src=https://i.turner.ncaa.com/sites/defau...,Sr.,RB,11,91,2,370
248,199,Willie Miller,<img src=https://i.turner.ncaa.com/sites/defau...,So.,QB,10,79,2,367


In [66]:
bigtable_fcs.to_csv('FCS_bigTable.csv')